# Import Libraries

In [9]:
from google.colab import files
import re

# Rule Class Definition

In [10]:
class Rule:
    """Class to represent a production rule"""
    def __init__(self, antecedents, consequent):
        self.antecedents = antecedents  # List of conditions (ANDed together)
        self.consequent = consequent    # Resulting fact


# File Parsing Functions

In [11]:
def parse_rules(content):
    """Parse the rules from text content"""
    rules = []
    for line in content.split('\n'):
        line = line.strip()
        if not line or line.startswith('#'):
            continue

        # Split into IF and THEN parts
        if ' THEN ' not in line:
            continue

        if_part, then_part = line.split(' THEN ', 1)
        if_part = if_part.replace('IF ', '').strip()

        # Handle OR conditions by splitting into separate rules
        if ' OR ' in if_part:
            or_parts = [p.strip() for p in if_part.split(' OR ')]
            for part in or_parts:
                antecedents = [a.strip() for a in part.split(' AND ')]
                rules.append(Rule(antecedents, then_part.strip()))
        else:
            antecedents = [a.strip() for a in if_part.split(' AND ')]
            rules.append(Rule(antecedents, then_part.strip()))
    return rules

def parse_facts(content):
    """Parse the initial facts from text content"""
    facts = {}
    for line in content.split('\n'):
        line = line.strip()
        if not line or line.startswith('#'):
            continue

        if '=' in line:
            key, value = line.split('=', 1)
            facts[key.strip()] = value.strip()
        else:
            facts[line] = True

    return facts

# Condition Evaluation Function

In [12]:
def evaluate_condition(condition, facts):
    """Evaluate if a condition is true given current facts"""
    if '=' in condition:
        key, val = condition.split('=', 1)
        return facts.get(key.strip()) == val.strip()
    elif '>' in condition:
        var, val = condition.split('>', 1)
        var = var.strip()
        return var in facts and facts[var].isdigit() and int(facts[var]) > int(val.strip())
    elif '<' in condition:
        var, val = condition.split('<', 1)
        var = var.strip()
        return var in facts and facts[var].isdigit() and int(facts[var]) < int(val.strip())
    else:
        return condition in facts

# Forward Chaining Algorithm

In [13]:
def forward_chaining(rules, initial_facts):
    """Implement forward chaining algorithm"""
    facts = initial_facts.copy()
    new_facts = True
    cycle = 0

    print("\n=== Forward Chaining Process ===")

    while new_facts:
        new_facts = False
        cycle += 1
        print(f"\nCycle {cycle} Facts: {sorted(facts.keys())}")

        for rule in rules:
            # Check if all antecedents are satisfied
            all_conditions_met = all(evaluate_condition(ant, facts) for ant in rule.antecedents)

            if all_conditions_met:
                consequent = rule.consequent
                if consequent not in facts:
                    facts[consequent] = True
                    new_facts = True
                    print(f"  Inferred new fact: {consequent}")
    return facts

# Backward Chaining Algorith

In [14]:
def backward_chaining(goal, rules, facts, depth=0, explored=None):
    """Implement backward chaining algorithm"""
    if explored is None:
        explored = set()

    print("  " * depth + f"Checking goal: {goal}")

    # Base case: goal is already in facts
    if evaluate_condition(goal, facts):
        print("  " * depth + f"Found in facts: {goal}")
        return True

    # Avoid cycles
    if goal in explored:
        print("  " * depth + f"Already explored: {goal}")
        return False

    explored.add(goal)

    # Find all rules that can conclude this goal
    relevant_rules = [rule for rule in rules if rule.consequent == goal]

    for rule in relevant_rules:
        print("  " * depth + f"Trying rule: IF {' AND '.join(rule.antecedents)} THEN {rule.consequent}")

        # Check if all antecedents can be proven
        all_proven = True
        for ant in rule.antecedents:
            if not backward_chaining(ant, rules, facts, depth+1, explored):
                all_proven = False
                break

        if all_proven:
            print("  " * depth + f"Proved all antecedents for: {goal}")
            return True

    print("  " * depth + f"Could not prove: {goal}")
    return False

# Main Execution

In [15]:
def main():
    # Upload files
    print("Please upload rules.txt and facts.txt")
    uploaded = files.upload()

    # Read and parse files
    with open('rules.txt', 'r') as f:
        rules = parse_rules(f.read())

    with open('facts.txt', 'r') as f:
        initial_facts = parse_facts(f.read())

    # Process diameter to determine size
    diameter = initial_facts.get('diameter')
    if diameter and diameter.isdigit():
        d = int(diameter)
        if d < 2:
            initial_facts['size is small'] = True
        elif d > 10:
            initial_facts['size is big'] = True
        else:
            initial_facts['size is medium'] = True

    print("\n=== Initial Facts ===")
    print(initial_facts)

    # Forward Chaining
    print("\n=== Starting Forward Chaining ===")
    final_facts = forward_chaining(rules, initial_facts)

    print("\n=== Forward Chaining Results ===")
    print("Final Facts:", sorted(final_facts.keys()))
    print("citrus_fruit proven:", 'citrus_fruit' in final_facts)

    # Backward Chaining (using initial facts)
    print("\n=== Starting Backward Chaining ===")
    goal = 'citrus_fruit'
    print(f"\nAttempting to prove goal: {goal}")
    result = backward_chaining(goal, rules, initial_facts)
    print(f"\n=== Backward Chaining Result ===")
    print(f"Goal '{goal}' is {'proven' if result else 'not proven'}")

if __name__ == "__main__":
    main()

Please upload rules.txt and facts.txt


Saving rules.txt to rules (1).txt
Saving facts.txt to facts (1).txt

=== Initial Facts ===
{'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color is orange': True, 'size is medium': True}

=== Starting Forward Chaining ===

=== Forward Chaining Process ===

Cycle 1 Facts: ['color is orange', 'diameter', 'seeds', 'size is medium', 'skin_smell']
  Inferred new fact: perfumed
  Inferred new fact: fruit is orange

Cycle 2 Facts: ['color is orange', 'diameter', 'fruit is orange', 'perfumed', 'seeds', 'size is medium', 'skin_smell']
  Inferred new fact: citrus_fruit

Cycle 3 Facts: ['citrus_fruit', 'color is orange', 'diameter', 'fruit is orange', 'perfumed', 'seeds', 'size is medium', 'skin_smell']

=== Forward Chaining Results ===
Final Facts: ['citrus_fruit', 'color is orange', 'diameter', 'fruit is orange', 'perfumed', 'seeds', 'size is medium', 'skin_smell']
citrus_fruit proven: True

=== Starting Backward Chaining ===

Attempting to prove goal: citrus_fruit
Checking goal: citrus_fr